# Preliminary Notebook

## 1. Import and Install Dependencies

In [118]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [119]:
import cv2 # opencv
import numpy as np
import os # easier file path handling
from matplotlib import pyplot as plt # im.show for easy visualization
import time # to insert "sleep" in between frames
import mediapipe as mp # for accessing and reading from webcam

## 6. Preprocess Data and Create Labels and Features

In [120]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [121]:
# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_Data_Jin-Ho')

In [122]:
# create label map (dict, our .json file)
label_map = {label:num for num, label in enumerate(actions)}

In [123]:
sequences, labels = [], [] # sequences will be x data, labels will be y data
# loop over all actions (words)
for action in actions: 
    # loop over all sequences (videos)
    for sequence in range(no_sequences): 
        window = [] # represents all frames of particular sequence (video)
        # loop through each frame
        for frame_num in range(sequence_length): 
            # load up current frame (frame_num)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) # append to one video
        sequences.append(window) # append all videos to sequence (for a word)
        labels.append(label_map[action])

In [124]:
X = np.array(sequences) # shape: (90, 30, 1662)

In [125]:
y = to_categorical(labels).astype(int) # one-hot encoded labels (words)
# shape: (90, 3)

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## 7. Build and Train LSTM Neural Network

In [127]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [128]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [129]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [130]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

In [131]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.

Epoch 1/2000
3/3 [==============================] - 3s 50ms/step - loss: 1.4101 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 0s 50ms/step - loss: 5.3390 - categorical_accuracy: 0.3294
Epoch 3/2000
3/3 [==============================] - 0s 42ms/step - loss: 1.8938 - categorical_accuracy: 0.2353
Epoch 4/2000
3/3 [==============================] - 0s 53ms/step - loss: 2.1884 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 0s 48ms/step - loss: 6.9532 - categorical_accuracy: 0.3294
Epoch 6/2000
3/3 [==============================] - 0s 43ms/step - loss: 9.6743 - categorical_accuracy: 0.2941
Epoch 7/2000
3/3 [==============================] - 0s 40ms/step - loss: 16.6250 - categorical_accuracy: 0.2235
Epoch 8/2000
3/3 [==============================] - 0s 40ms/step - loss: 14.7669 - categorical_accuracy: 0.2941
Epoch 9/2000
3/3 [==============================] - 0s 38ms/step - loss: 18.2139 - categorical_accuracy: 0.341

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir=./Logs/train --port=8008
# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008